In [1]:
import pandas as pd
import json
import os
import time
import pickle
import ast
from matplotlib.backends.backend_pdf import PdfPages

In [4]:
root_dirs_dict = {"esol":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/esol/65",
                             "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/85",
                         "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/94"],
                 "BBBP":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/BBBP/64",
                             "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/85",
                        "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/94"],
                 "Lipophilicity":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/Lipophilicity/66",
                                  "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/85",
                                 "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/94"],
                 "Mutagenicity":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/Mutagenicity/73",
                                  "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/85",
                                "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/94"],
                  "hERG":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/hERG/69",
                                  "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/85",
                                "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/94"]}

In [5]:
# root_dirs_dict = {"Mutagenicity":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/Mutagenicity/73",
#                                   "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/85"]}

In [6]:
import os
import json
import pandas as pd
import re
import pdb

# root_dir = "/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/58"
# root_dirs_dict = {"Lipophilicity":["/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/60",
#                               "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/61",
#                               "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/63"
#                              ],
#                       "esol":["/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/60",
#                              "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/62"]}


# Architectures and types to check
architectures = ["GATConv", "GINConv", "GCNConv"]
types = ["MGSSL", "RBRICS","None"]

# Folder name pattern: EXPT-{number}R-{dataset}-{seed}-{fold}-{architecture}-{type}
folder_pattern = re.compile(
    r"EXPT-\d+[a-zA-Z]+-[a-zA-Z]+-SEED-\d+-FOLD-\d+-(GATConv|GINConv|GCNConv)-[^-]+-(MGSSL|RBRICS)"
)

folder_pattern_vanilla = re.compile(
    r"EXPT-\d+[a-zA-Z]+-[a-zA-Z]+-SEED-\d+-FOLD-\d+-(GATConv|GINConv|GCNConv)-[^-]+-(None)?"
)

for dataset_name, list_dirs in root_dirs_dict.items():
    # Initialize the results dictionary
    results = {
        "Architecture": [],
        "Type": [],
        "Train Metric": [],
        "Validation Metric": [],
        "Test Metric": [],
    }
    

    for root_dir in list_dirs:
        # Traverse the directory and process matching folders
        for folder in os.listdir(root_dir):
            if folder_pattern.match(folder) or folder_pattern_vanilla.match(folder):
                # Extract architecture and type from the folder name
                match = folder_pattern.match(folder)
                if match is not None:
                    architecture, model_type = match.groups()
                else:
                    match = folder_pattern_vanilla.match(folder)
                    architecture, model_type = match.groups()

                if architecture in architectures and model_type in types:
                    # Check for the JSON file
                    file_path = os.path.join(root_dir, folder, f"{dataset_name}_classification_result.json")
                    if os.path.exists(file_path):
                        with open(file_path, "r") as f:
                            data = json.load(f)
                        

                        # Determine the appropriate metrics based on dataset_name
                        if dataset_name in ["BBBP","Mutagenicity","hERG"]:
                            train_metric = data.get("Trained_explainations_train_rocauc", 0)
                            validation_metric = data.get("Trained_explainations_validation_rocauc", 0)
                            test_metric = data.get("Trained_explainations_test_rocauc", 0)
                        else:
                            train_metric = data.get("Trained_explainations_train_rmse", 0)
                            validation_metric = data.get("Trained_explainations_validation_rmse", 0)
                            test_metric = data.get("Trained_explainations_test_rmse", 0)

                        # Append single values for the current folder
                        results["Architecture"].append(architecture)
                        results["Type"].append(model_type)
                        results["Train Metric"].append(train_metric)
                        results["Validation Metric"].append(validation_metric)
                        results["Test Metric"].append(test_metric)

    # Create a DataFrame
    df = pd.DataFrame(results)

    df.to_csv(f"../kdd_results/results_summary_{dataset_name}.csv", index=False)

    # Group by Architecture and Type, calculate mean and standard deviation
    grouped = df.groupby(["Architecture", "Type"]).agg(
        Train_Mean=("Train Metric", "mean"),
        Train_Std=("Train Metric", "std"),
        Validation_Mean=("Validation Metric", "mean"),
        Validation_Std=("Validation Metric", "std"),
        Test_Mean=("Test Metric", "mean"),
        Test_Std=("Test Metric", "std"),
    ).reset_index()

    # Save to CSV
    output_csv = f"../kdd_results/results_summary_grouped_{dataset_name}.csv"
    grouped.to_csv(output_csv, index=False)

    print("Grouped results saved to:", output_csv)
    print(grouped)



Grouped results saved to: ../kdd_results/results_summary_grouped_esol.csv
  Architecture    Type  Train_Mean  Train_Std  Validation_Mean   
0      GATConv   MGSSL    0.434238   0.030430         0.464212  \
1      GATConv    None    0.455743   0.016453         0.470270   
2      GATConv  RBRICS    0.407901   0.021031         0.438115   
3      GCNConv   MGSSL    0.440523   0.031269         0.455010   
4      GCNConv    None    0.455685   0.034852         0.477031   
5      GCNConv  RBRICS    0.390118   0.014134         0.416379   
6      GINConv   MGSSL    0.432184   0.048853         0.471568   
7      GINConv    None    0.433375   0.032411         0.488646   
8      GINConv  RBRICS    0.403754   0.042286         0.476240   

   Validation_Std  Test_Mean  Test_Std  
0        0.042040   0.455019  0.046591  
1        0.029702   0.469885  0.038958  
2        0.032206   0.451754  0.050153  
3        0.046624   0.453239  0.045117  
4        0.034424   0.465019  0.050968  
5        0.034225  

In [7]:
# root_dirs_dict = {"esol":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/esol/65"],
#                  "BBBP":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/BBBP/64"],
#                  "Lipophilicity":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/Lipophilicity/66"],
#                  "Mutagenicity":["/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/KDD/Mutagenicity/73"]}

In [8]:
# Show casing the importanve vs logit diff per epoch
import os
import json
import pandas as pd
import re
import pdb
import torch
import ast
import matplotlib.pyplot as plt
from DataLoader import get_setup_files_with_folds

# root_dir = "/nfs/stak/users/kokatea/hpc-share/ChemIntuit/Cluster_JOBS/Regression/58"
# root_dirs_dict = {"Lipophilicity":["/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/61",
#                               "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/63"
#                              ],
#                       "esol":["/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/62"]}

# Architectures and types to check
architectures = ["GATConv", "GINConv", "GCNConv"]
types = ["MGSSL", "RBRICS"]

# Define folder name patterns
folder_pattern = re.compile(
    r"EXPT-\d+[a-zA-Z]+-[a-zA-Z]+-SEED-(\d+)-FOLD-(\d+)-(GATConv|GINConv|GCNConv)-[^-]+-(MGSSL|RBRICS)"
)

date_tag = '1225'
use_count = True

# Loop through datasets and directories
for dataset_name, list_dirs in root_dirs_dict.items():
    pdf_path = f"../kdd_results/{dataset_name}_counts_{use_count}plots.pdf"
    with PdfPages(pdf_path) as pdf:
        root_dir = list_dirs[0]
        # for root_dir in list_dirs:
        # Dictionary to track data availability
        plot_data = {arch: {type_: {} for type_ in types} for arch in architectures}

        # Traverse the directory and process matching folders
        for folder in os.listdir(root_dir):
            if folder_pattern.match(folder):
                # Extract architecture, type, seed, and fold from the folder name
                match = folder_pattern.match(folder)
                seed, fold, architecture, model_type = match.groups()

                if architecture in architectures and model_type in types:
                    # Check for the CSV file
                    file_path = os.path.join(root_dir, folder, f"explainer/{dataset_name}.csv")
                    if os.path.exists(file_path):
                        # Read the CSV file
                        df = pd.read_csv(file_path)


                        # Process `Logit Diff` and `Motif params`
                        df['Logit Diff'] = df['Logit Diff'].apply(ast.literal_eval)
                        df['Motif params'] = df['Motif params'].apply(lambda x: torch.sigmoid(torch.tensor(ast.literal_eval(x))).tolist())

                        # Fetch `motif_lengths` from setup files
                        _, _, motif_counts, motif_lengths, _, _, _, _, _, _, _ = get_setup_files_with_folds(dataset_name, date_tag, fold, model_type)



                       # Store data for plotting by seed and fold
                        plot_data[architecture][model_type].setdefault(seed, {})[fold] = (df, motif_lengths,motif_counts)

            
            # Rename architectures
            architecture_names = {"GATConv": "GAT", "GINConv": "GIN", "GCNConv": "GCN"}

            # Iterate over seeds and folds for plotting
            for seed in [0]:
                for fold in range(5):  # Assuming 0-4 folds
                    # Set up the plot
                    fig, axes = plt.subplots(2, len(architectures), figsize=(20, 10))
                    fig.suptitle(f"{dataset_name} - Seed {seed} Fold {fold}", fontsize=16)
                    plt.subplots_adjust(hspace=0.6, wspace=0.6)
                    
                    avg_motifs = {}

                    # Plot MGSSL and RDBRICS on separate rows
                    for i, arch in enumerate(architectures):
                        for j, model_type in enumerate(types):
                            ax = axes[j, i]
                            # pdb.set_trace()
                            plot_data_entry = plot_data[arch][model_type].get(str(seed), {}).get(str(fold), None)
                            if plot_data_entry is not None:
                                df, motif_lengths,motif_counts = plot_data_entry
                                # pdb.set_trace()
                                num_motifs = len(df['Motif params'][0])
                                for motif_idx in range(num_motifs):
                                    try:
                                        # Extract start, trajectory, and end points
                                        start_x, start_y = df['Motif params'][0][motif_idx], df['Logit Diff'][0][motif_idx]
                                        end_x, end_y = df['Motif params'].iloc[-1][motif_idx], df['Logit Diff'].iloc[-1][motif_idx]
                                        trajectory = [
                                            (epoch[motif_idx], diff[motif_idx])
                                            for epoch, diff in zip(df['Motif params'], df['Logit Diff']) if len(epoch) > motif_idx
                                        ]
                                        traj_x, traj_y = zip(*trajectory)
                                        
                                        if use_count:
                                            # Highlight start and end points with color based on motif length
                                            motif_count = list(motif_counts.values())[motif_idx]
                                            color = plt.cm.tab20c(motif_count / max(list(motif_counts.values())))
                                        else:
                                            # Highlight start and end points with color based on motif length
                                            motif_length = list(motif_lengths.values())[motif_idx]
                                            color = plt.cm.winter(motif_length / max(list(motif_lengths.values())))


                                        # Plot trajectory
                                        ax.plot(traj_x, traj_y, color=color, alpha=0.7)
                                        
                                        # Ensure points are visible with a minimum size
                                        point_size = 50
                                        
                                        ax.scatter([start_x], [start_y], color='green', s=20)
                                        ax.scatter([end_x], [end_y], color=color, s=20)

                                        # Plot start and end points with color
                                        # ax.scatter([start_x], [start_y], color=color, s=point_size, edgecolor='black', label="Start")
                                        # ax.scatter([end_x], [end_y], color=color, s=point_size, edgecolor='black', label="End")

                                        # ax.scatter([start_x], [start_y], color='green', s=point_size, label="Start")
                                        # ax.scatter([end_x], [end_y], color=color, s=point_size, edgecolor='black', label="End")
                                        # ax.scatter([start_x], [start_y], color='green', s=20, label="Start")
                                        # ax.scatter([end_x], [end_y], color='red', s=20, label="End")
                                        
                                    except (IndexError, KeyError):
                                        continue
                                        
                                # Add epoch count annotation
                                num_epochs = len(df)
                                ax.text(0.35, 0.15, f"Epochs: {num_epochs}",
                                        transform=ax.transAxes, fontsize=10,
                                        ha='right', va='top', bbox=dict(facecolor='white', alpha=0.8))

                                # Set plot labels
                                ax.set_title(f"{architecture_names[arch]} - {model_type}", fontsize=12)
                                ax.set_xlabel("Motif Parameters")
                                ax.set_ylabel("Old Logit - Logit after removal")
                                ax.set_xlim(0, 1)  # Set x-axis between 0 and 1
                                ax.grid(True)
                            else:
                                # Add blank plot for missing data
                                ax.set_title(f"{architecture_names[arch]} - {model_type}", fontsize=12)
                                ax.text(0.5, 0.5, "No Data", fontsize=16, ha='center', va='center', color='gray')
                                ax.set_xticks([])
                                ax.set_yticks([])
                                ax.set_frame_on(False)
                    if use_count:
                        # Add colorbar for end points (shared across plots)
                        sm = plt.cm.ScalarMappable(cmap='tab20c', norm=plt.Normalize(vmin=min(list(motif_counts.values())), vmax=max(list(motif_counts.values()))))
                        sm.set_array([])
                        fig.colorbar(sm, ax=axes[:, :], orientation='vertical', label='Motif Frequency')
                    else:
                        # Add colorbar for end points (shared across plots)
                        sm = plt.cm.ScalarMappable(cmap='winter', norm=plt.Normalize(vmin=min(list(motif_lengths.values())), vmax=max(list(motif_lengths.values()))))
                        sm.set_array([])
                        fig.colorbar(sm, ax=axes[:, :], orientation='vertical', label='Motif Length')


                    # Save the figure to the PDF
                    pdf.savefig(fig)
                    plt.close(fig)
            break

    print(f"PDF saved: {pdf_path}")



# # Loop through datasets and directories
# for dataset_name, list_dirs in root_dirs_dict.items():
#     for root_dir in list_dirs:
#         # Traverse the directory and process matching folders
#         for folder in os.listdir(root_dir):
#             if folder_pattern.match(folder):
#                 # Extract architecture and type from the folder name
#                 match = folder_pattern.match(folder)

#                 # Extract information
#                 seed, fold, architecture, model_type = match.groups()

#                 if architecture in architectures and model_type in types:
#                     # Check for the CSV file
#                     file_path = os.path.join(root_dir, folder, f"explainer/{dataset_name}.csv")
#                     if os.path.exists(file_path):
#                         # Read the CSV file
#                         df = pd.read_csv(file_path)

#                         # Process `Logit Diff` and `Motif params`
#                         df['Logit Diff'] = df['Logit Diff'].apply(ast.literal_eval)
#                         df['Motif params'] = df['Motif params'].apply(ast.literal_eval)

#                         # Plot the data
#                         num_epochs = len(df)
#                         num_motifs = len(df['Motif params'][0])  # Assuming all rows have consistent motif counts

#                         plt.figure(figsize=(12, 8))

#                         for motif_idx in range(num_motifs):
#                             try:
#                                 # Extract start, trajectory, and end points for each motif
#                                 start_x, start_y = df['Motif params'][0][motif_idx], df['Logit Diff'][0][motif_idx]
#                                 end_x, end_y = df['Motif params'].iloc[-1][motif_idx], df['Logit Diff'].iloc[-1][motif_idx]
#                                 trajectory = [
#                                     (epoch[motif_idx], diff[motif_idx])
#                                     for epoch, diff in zip(df['Motif params'], df['Logit Diff']) if len(epoch) > motif_idx
#                                 ]

#                                 # Extract X and Y coordinates for trajectory
#                                 traj_x, traj_y = zip(*trajectory)

#                                 # Plot trajectory
#                                 plt.plot(traj_x, traj_y, alpha=0.5, label=f"Motif {motif_idx} Trajectory")

#                                 # Highlight start and end points
#                                 plt.scatter([start_x], [start_y], color='green', label=f"Motif {motif_idx} Start")
#                                 plt.scatter([end_x], [end_y], color='red', label=f"Motif {motif_idx} End")
#                             except (IndexError, KeyError):
#                                 print(f"Error processing motif {motif_idx}, skipping...")
#                                 continue

#                         # Enhance visualization
#                         plt.title(f"Motif Trajectories for {dataset_name} ({architecture} - {model_type})")
#                         plt.xlabel("Motif Parameters")
#                         plt.ylabel("Old Logit - New logit (Motif removed)")
#                         # plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
#                         plt.tight_layout()

#                         # Show the plot
#                         plt.show()
# for dataset_name, list_dirs in root_dirs_dict.items():
#     for root_dir in list_dirs:
#         # Traverse the directory and process matching folders
#         for folder in os.listdir(root_dir):
#             if folder_pattern.match(folder) :
#                 # Extract architecture and type from the folder name
#                 match = folder_pattern.match(folder)

#                 # Extract information
#                 seed, fold, architecture, model_type = match.groups()

#                 if architecture in architectures and model_type in types:
#                     # Check for the CSV file
#                     file_path = os.path.join(root_dir, folder, f"explainer/{dataset_name}.csv")
#                     if os.path.exists(file_path):
#                         # Read the CSV file
#                         df = pd.read_csv(file_path)
                        
#                         # Process `Logit Diff` and `Motif params`
#                         df['Logit Diff'] = df['Logit Diff'].apply(ast.literal_eval)
#                         df['Motif params'] = df['Motif params'].apply(ast.literal_eval)

#                         # Flatten the `Logit Diff` and `Motif params` for plotting
#                         logit_diffs = []
#                         motif_params = []
#                         for _, row in df.iterrows():
#                             logit_diffs.extend(row['Logit Diff'].values())
#                             motif_params.extend(row['Motif params'])

#                         # Plot the data
#                         plt.figure(figsize=(10, 6))
#                         plt.scatter(motif_params, logit_diffs, alpha=0.6, edgecolors='w')
#                         plt.xlabel('Motif Parameters')
#                         plt.ylabel('Logit Diff')
#                         plt.title(f'Logit Diff vs Motif Parameters ({dataset_name} - {architecture} - {model_type})')
#                         plt.grid(True)
#                         plt.show()
                            
                


PDF saved: ../kdd_results/esol_counts_Trueplots.pdf
PDF saved: ../kdd_results/BBBP_counts_Trueplots.pdf
PDF saved: ../kdd_results/Lipophilicity_counts_Trueplots.pdf
PDF saved: ../kdd_results/Mutagenicity_counts_Trueplots.pdf
PDF saved: ../kdd_results/hERG_counts_Trueplots.pdf


In [41]:
import os
import pandas as pd

def process_results_summary(folder_path):
    # Initialize a list to store processed data for all datasets
    processed_data = []

    # Iterate over all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file matches the required pattern
        if file_name.startswith("results_summary_grouped_") and file_name.endswith(".csv"):
            # Extract the dataset name from the file name
            dataset_name = file_name[len("results_summary_grouped_"):-len(".csv")]

            # Read the CSV file into a DataFrame
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)

            # Replace 'None' with 'Vanilla' in the 'Type' column
            df['Type'] = df['Type'].fillna('Vanilla')
            
            # Replace architecture values
            df['Architecture'] = df['Architecture'].replace({
                "GATConv": "GAT",
                "GINConv": "GIN",
                "GCNConv": "GCN"
            })

            # Format Train, Validation, and Test metrics to 0.3f precision
            df['Train Metric'] = df['Train_Mean'].map("{:.3f}".format) + " ± " + df['Train_Std'].map("{:.3f}".format)
            df['Validation Metric'] = df['Validation_Mean'].map("{:.3f}".format) + " ± " + df['Validation_Std'].map("{:.3f}".format)
            df['Test Metric'] = df['Test_Mean'].map("{:.3f}".format) + " ± " + df['Test_Std'].map("{:.3f}".format)

            # Keep only relevant columns
            df = df[['Architecture', 'Type', 'Train Metric', 'Validation Metric', 'Test Metric']]

            # Add the dataset name to the DataFrame
            df.insert(0, 'Dataset Name', dataset_name)
            
            # Sort the DataFrame by 'Type' with 'Vanilla' first
            df['Type'] = pd.Categorical(df['Type'], categories=['Vanilla', 'MGSSL','RBRICS'], ordered=True)
            df['Architecture'] = pd.Categorical(df['Architecture'], categories=['GAT', 'GCN','GIN'], ordered=True)
            
            df = df.sort_values(by=['Dataset Name', 'Architecture', 'Type'])
            
            

            # Append to the processed data list
            processed_data.append(df)

    # Concatenate all processed data into a single DataFrame
    final_df = pd.concat(processed_data, ignore_index=True)

    # Split the data into two groups based on the dataset
    group_1_datasets = ["esol", "Lipophilicity"]
    group_2_datasets = ["Mutagenicity", "BBBP", "hERG"]

    group_1_df = final_df[final_df['Dataset Name'].isin(group_1_datasets)].copy()
    group_2_df = final_df[final_df['Dataset Name'].isin(group_2_datasets)].copy()

    # Function to generate LaTeX table for a group
    def generate_latex_table(dataframe, output_file):
        # Remove duplicate dataset names for multirow formatting
        dataframe.loc[:, 'Dataset Name'] = dataframe['Dataset Name'].mask(dataframe['Dataset Name'].duplicated(), '')

        # Convert DataFrame to LaTeX table
        latex_code = dataframe.to_latex(index=False, escape=False, column_format="|l|l|l|l|l|l|", header=[
            "Dataset Name", "GNN Architecture", "Model", "Train Metric", "Validation Metric", "Test Metric"
        ])
        
        # Add table borders and formatting
        latex_code = latex_code.replace('\\toprule', '\\hline').replace('\\midrule', '').replace('\\bottomrule', '\\hline')

        # Write the LaTeX code to a file
        with open(output_file, "w") as f:
            f.write(latex_code)

        print(f"LaTeX table saved to {output_file}")

    # Generate LaTeX tables for the two groups
    group_1_file = os.path.join(folder_path, "results_summary_regression.tex")
    group_2_file = os.path.join(folder_path, "results_summary_binary_classification.tex")

    generate_latex_table(group_1_df, group_1_file)
    generate_latex_table(group_2_df, group_2_file)


In [42]:
# Example usage
folder_path = "../kdd_results"  # Replace with the path to your folder
process_results_summary(folder_path)

LaTeX table saved to ../kdd_results/results_summary_regression.tex
LaTeX table saved to ../kdd_results/results_summary_binary_classification.tex


In [4]:
import os
import json
import pandas as pd
import re

# Root directory containing the folders
root_dir = "/nfs/stak/users/kokatea/ondemand/data/sys/myjobs/projects/default/54"

# Initialize data structure to store results
results = {
    "Architecture": [],
    "Type": [],
    "Train Mean": [],
    "Train Std": [],
    "Validation Mean": [],
    "Validation Std": [],
    "Test Mean": [],
    "Test Std": []
}

# Architectures and types to check
architectures = ["GATConv", "GINConv", "GCNConv"]
types = ["MGSSL", "RBRICS"]

# Folder name pattern: EXPT-{number}R-{dataset}-{seed}-{fold}-{architecture}-{type}
folder_pattern = re.compile(
    r"EXPT-\d+R-[a-zA-Z]+-SEED-\d+-FOLD-\d+-(GATConv|GINConv|GCNConv)-[^-]+-(MGSSL|RBRICS)"
)
# EXPT-12R-esol-SEED-0-FOLD-0-GATConv-SingleChannel-RBRICS
# Traverse the directory and process matching folders
for folder in os.listdir(root_dir):

    if folder_pattern.match(folder):

        # Extract architecture and type from the folder name
        match = folder_pattern.match(folder)
        architecture, model_type = match.groups()

        if architecture in architectures and model_type in types:
            train_maes, val_maes, test_maes = [], [], []

            file_path = os.path.join(root_dir, folder, "esol_classification_result.json")
            if os.path.exists(file_path):
                with open(file_path, "r") as f:
                    data = json.load(f)

                train_maes.append(data.get("Trained_explainations_train_mae", 0))
                val_maes.append(data.get("Trained_explainations_validation_mae", 0))
                test_maes.append(data.get("Trained_explainations_test_mae", 0))

            # Calculate statistics for the current architecture and type
            train_mean = sum(train_maes) / len(train_maes) if train_maes else 0
            train_std = pd.Series(train_maes).std() if train_maes else 0
            val_mean = sum(val_maes) / len(val_maes) if val_maes else 0
            val_std = pd.Series(val_maes).std() if val_maes else 0
            test_mean = sum(test_maes) / len(test_maes) if test_maes else 0
            test_std = pd.Series(test_maes).std() if test_maes else 0

            # Append results
            results["Architecture"].append(architecture)
            results["Type"].append(model_type)
            results["Train Mean"].append(train_mean)
            results["Train Std"].append(train_std)
            results["Validation Mean"].append(val_mean)
            results["Validation Std"].append(val_std)
            results["Test Mean"].append(test_mean)
            results["Test Std"].append(test_std)

# Create a DataFrame and display the results
df = pd.DataFrame(results)
print(df)
merged_df = df.groupby(["Architecture", "Type"], as_index=False).agg({
    "Train Mean": "mean",
    "Train Std": "mean",  # Mean of stds across folds/seeds
    "Validation Mean": "mean",
    "Validation Std": "mean",
    "Test Mean": "mean",
    "Test Std": "mean"
})
# Optionally, save to a CSV file
df.to_csv("results_summary.csv", index=False)


   Architecture    Type  Train Mean  Train Std  Validation Mean  \
0       GINConv  RBRICS    3.202261        NaN         3.244942   
1       GCNConv  RBRICS    3.143503        NaN         3.360423   
2       GATConv  RBRICS    3.133229        NaN         3.359058   
3       GATConv  RBRICS    3.232798        NaN         2.991389   
4       GINConv   MGSSL    3.215844        NaN         3.254110   
5       GATConv   MGSSL    3.151217        NaN         3.366590   
6       GCNConv   MGSSL    3.116649        NaN         3.503303   
7       GCNConv   MGSSL    3.285963        NaN         3.162374   
8       GCNConv  RBRICS    3.301915        NaN         3.179567   
9       GATConv   MGSSL    3.274320        NaN         3.154436   
10      GCNConv   MGSSL    3.281859        NaN         3.316725   
11      GINConv   MGSSL    3.139627        NaN         3.531137   
12      GINConv   MGSSL    3.217353        NaN         3.107682   
13      GATConv   MGSSL    3.103157        NaN         3.50518

In [5]:
print(folder_pattern.match(folder))

None


In [6]:
'''
Change result dict when you run new experiments 

folder - Enter list of folders to check, each folder will be traversed to check if file exists in folder
seed
arch
size_reg
dataset

**IMPORTANT** : Change EXPT_BASE_NAME if you update the folder name to where your results are stored


results_dict = {
    
    'folder' : [38,39,40,41],
    'seed' : [2,3],
    'arch' : ['GINConv','GCNConv','GATConv'],
    'size_reg' : [0,0.001,0.1],
    'dataset'  : {'BBBP':'BBBP','MUTAG':'Mutagenicity','HERG':'hERG'}
}


results_dict = {
    
    'folder' : [27,29,30,31,32,33,34,35],
    'seed' : [0,1],
    'arch' : ['GINConv','GCNConv','GATConv'],
    'size_reg' : [0,0.001,0.1],
    'dataset'  : {'BBBP':'BBBP','MUTAG':'Mutagenicity','HERG':'hERG'}
}


results_dict = {
    
    'folder' : [42,43],
    'seed' : [2,3],
    'arch' : ['GINConv','GCNConv','GATConv'],
    'channel' : ['Vanilla'],
    'dataset'  : {'BBBP':'BBBP','MUTAG':'Mutagenicity'}
}

'''
date_tag = '0830'


expt_dict = [
    
    {"dataset":"HERG", "seed_info" : {1:49,3:49,0:48,2:48}},
    {"dataset":"MUTAG", "seed_info" : {1:47,3:47,0:46,2:46}},
    {"dataset":"BBBP", "seed_info" : {1:47,3:47,0:46,2:46}}]

dataset_dict = {'BBBP':'BBBP','MUTAG':'Mutagenicity','HERG':'hERG'}
#dataset_dict = {'BBBP':'BBBP'}

def return_expt_base_name(seed,layer_type,channel,folder,dataset):

    BASE_PATH = f'../Cluster_JOBS/LoG_plot/{folder}/EXPT-32A{dataset}-{seed}-{layer_type}-{channel}'
    return BASE_PATH

In [7]:
result_list = []

print(f'Adding the Following Config to Final Results, copy them over to add to the plots pdf')

motif_count_dict = {}

for dataset in dataset_dict.keys():
    with open(f'dictionary/{dataset_dict[dataset]}_motif_list_{date_tag}.pickle', 'rb') as file:
        motif_list = list(pickle.load(file))
    if dataset not in motif_count_dict.keys():
        motif_count_dict[dataset] = {}
    
    motif_count_dict[dataset]['model'] = {}
    motif_count_dict[dataset]['logit'] = {}
        
        
    for channel in ["DualParam","SingleChannel"]:
        for layer_type in ["GINConv", "GCNConv", "GATConv"]:
            for seed in [0,1,2,3]:
                
                for folder_dict in expt_dict:
                    if folder_dict['dataset'] == dataset:
                        folder = folder_dict['seed_info'][seed]
                
                EXPT_PATH = return_expt_base_name(seed,layer_type,channel,folder,dataset)
                
                print(EXPT_PATH)
                
                found = True

                if os.path.exists(EXPT_PATH):

                    try:

                        #Recreate loss metrics
                        logit_val = pd.read_csv(EXPT_PATH+f'/{dataset_dict[dataset]}_explanation_result_with_validation.csv')
                        logit_test = pd.read_csv(EXPT_PATH+f'/{dataset_dict[dataset]}_explanation_result_with_test.csv')
                    except Exception as e:
                        print(f"error when trying to validation motif logit differences: {e}")
                        found = False
                    
                    #validation
                    logit_val['logit_diff_class_0'] = logit_val['new_logit_class_0'] - logit_val['original_logit_class_0']
                    logit_val['logit_diff_class_1'] = logit_val['new_logit_class_1'] - logit_val['original_logit_class_1']
                    
                    #test
                    logit_test['logit_diff_class_0'] = logit_test['new_logit_class_0'] - logit_test['original_logit_class_0']
                    logit_test['logit_diff_class_1'] = logit_test['new_logit_class_1'] - logit_test['original_logit_class_1']
                                        
                    if channel == 'DualParam':
                    
                        #get mean scores for val
                        logit_val = logit_val.groupby('motif_id').agg(
                                                        mean_logit_diff_class_0   = ('logit_diff_class_0', 'mean'),
                                                        mean_logit_diff_class_1   = ('logit_diff_class_1', 'mean'),
                                                        mean_sig_imp_class_0      = ('sigmoid_importance_for_class_0', 'mean'),
                                                        mean_sig_imp_class_1      = ('sigmoid_importance_for_class_1', 'mean')).reset_index()
                        
                        #get mean scores for test
                        logit_test = logit_test.groupby('motif_id').agg(
                                                        mean_logit_diff_class_0   = ('logit_diff_class_0', 'mean'),
                                                        mean_logit_diff_class_1   = ('logit_diff_class_1', 'mean'),
                                                        mean_sig_imp_class_0      = ('sigmoid_importance_for_class_0', 'mean'),
                                                        mean_sig_imp_class_1      = ('sigmoid_importance_for_class_1', 'mean')).reset_index()
                    else:                    
                        #get mean scores for val
                        logit_val = logit_val.groupby('motif_id').agg(
                                                        mean_logit_diff_class_0   = ('logit_diff_class_0', 'mean'),
                                                        mean_logit_diff_class_1   = ('logit_diff_class_1', 'mean'),
                                                        mean_sig_imp_class_0      = ('sigmoid_importance_for_class_0', 'mean')).reset_index()
                        
                        #get mean scores for test
                        logit_test = logit_test.groupby('motif_id').agg(
                                                        mean_logit_diff_class_0   = ('logit_diff_class_0', 'mean'),
                                                        mean_logit_diff_class_1   = ('logit_diff_class_1', 'mean'),
                                                        mean_sig_imp_class_0      = ('sigmoid_importance_for_class_0', 'mean')).reset_index()
                        
                    
                    # sort by logit_diff
                    logit_0_motifs = logit_val.sort_values('mean_logit_diff_class_0', ascending = False).motif_id.tolist()[0:10]
                    
                    for motif in logit_0_motifs:
                        if motif_list[motif] not in motif_count_dict[dataset]['logit'].keys():
                            motif_count_dict[dataset]['logit'][motif_list[motif]] = 1
                        else:
                            motif_count_dict[dataset]['logit'][motif_list[motif]] += 1
                    
                    logit_1_motifs = logit_val.sort_values('mean_logit_diff_class_1', ascending = False).motif_id.tolist()[0:10]
                    
                    for motif in logit_1_motifs:
                        if motif_list[motif] not in motif_count_dict[dataset]['logit'].keys():
                            motif_count_dict[dataset]['logit'][motif_list[motif]] = 1
                        else:
                            motif_count_dict[dataset]['logit'][motif_list[motif]] += 1
                            
                    
#                     #sort by model importance
#                     model_0_motifs = logit_val.sort_values('mean_sig_imp_class_0', ascending = False).motif_id.tolist()[0:10]
                    
#                     for motif in model_0_motifs:
#                         if motif_list[motif] not in motif_count_dict[dataset]['model'].keys():
#                             motif_count_dict[dataset]['model'][motif_list[motif]] = 1
#                         else:
#                             motif_count_dict[dataset]['model'][motif_list[motif]] += 1
                    
#                     if channel == 'DualParam':
#                         model_1_motifs = logit_val.sort_values('mean_sig_imp_class_1', ascending = False).motif_id.tolist()[0:10]
                    
#                         for motif in model_1_motifs:
#                             if motif_list[motif] not in motif_count_dict[dataset]['model'].keys():
#                                 motif_count_dict[dataset]['model'][motif_list[motif]] = 1
#                             else:
#                                 motif_count_dict[dataset]['model'][motif_list[motif]] += 1
                    
                            
print(motif_count_dict)

Adding the Following Config to Final Results, copy them over to add to the plots pdf


FileNotFoundError: [Errno 2] No such file or directory: 'dictionary/BBBP_motif_list_0830.pickle'

In [ ]:
# df = pd.read_csv(result_path,index_col = None)
# df['model_type']='DualParam'
# df.to_csv(result_path, index = False)

In [ ]:
rows = []

# Iterate through the dictionary
for dataset, models in motif_count_dict.items():
    for model, attributes in models.items():
        for key, value in attributes.items():
            # Append a tuple with the required structure
            rows.append((dataset, model, key, value))

# Create a DataFrame
df = pd.DataFrame(rows, columns=['Dataset', 'Importance Criteria', 'Motif Name', 'Count'])

df = df.sort_values(['Dataset', 'Importance Criteria', 'Count'], ascending = False)

# Display the DataFrame
df.to_csv('important_motif_count_logit.csv',index = False)